# Adversarial Search: Playing Connect 4


## Instructions

Total Points: Undegraduates 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play Connect 4:

> "Connect 4 is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs." (see [Connect Four on Wikipedia](https://en.wikipedia.org/wiki/Connect_Four))

## Task 1: Defining the Search Problem [1 point]

Define the components of the search problem:

* Initial state - The initial state is 
* Actions - A column number for where the player drops their playing piece. It will fall to the lowest open place for that column.
* Transition model - Place a piece in the column that was selected by the player
* Goal state - Create 4 pieces in a row



* **Initial state** - The initial state is 
* **Actions** - A column number for where the player drops their playing piece. It will fall to the lowest open place for that column.
* **Transition model** - Place a piece in the column that was selected by the player
* **Goal state** - Create 4 pieces in a row

How big is the search space?

TODO: calculate this state space

__Note:__ The search space for a $6 \times 7$ board is large. You can experiment with smaller boards (the smallest is $4 times \4$) and/or changing the winning rule to connect 3 instead of 4.

## Task 2: Game Environment and Random Agent [2 point]

Use a numpy character array as the board.

In [1]:
import numpy as np

def empty_board(shape=(6, 7)):
    return np.full(shape=shape, fill_value=' ')

def fill_board(board):
    s = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    i = 0
    for row in range(len(board)):
        for col in range(len(board[0])):
            board[row][col] = s[i%len(s)]
            i+=1

print(empty_board())

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']]


Instead of colors for the players use 'x' and 'o' to represent the players. Make sure that your agent functions all have the from: `agent_type(board, player = 'x')`, where board is the current board position and player is the player whose next move it is and who the agent should play.

Implement the board and helper functions for:

* The transition model (result).
* The utility function.
* Check for terminal states.
* A check for available actions.
* A function to visualize the board.

Make sure that all these functions work with boards of different sizes.

Implement an agent that plays randomly and let two random agents play against each other 1000 times. How often does each player win? Is the result expected? 

In [26]:
import random

def flip_player(player):
    if player == 'x':
        return 'o'
    else:
        return 'x'
    
def actions(board):
    options = []
    for col in range(0, len(board[0])):
        if board[0][col] == ' ':
            options.append(col)
    return options

def random_order_actions(board):
    options = actions(board)
    random.shuffle(options)
    return options

def agent_random(board, player = 'x'):
    options = actions(board)
    if len(options) == 0:
        return None
    return random.choice(options)
            
# Environment methods
def place_piece(board, col, player='x'):
    """Place the player piece in the col on the board"""
    if board[0][col] != ' ':
        return False
    
    next_open_row = len(board) - 1 # if the col is empty use the bottom row
    for row in range(0, len(board)):
        if board[row][col] != ' ':
            next_open_row = row - 1
            break
            
    board[next_open_row][col] = player


def result(board, player, action):
    board_copy = board.copy()
    place_piece(board_copy, action, player)
    return board_copy


# TODO: Optimize this
def check_series_for_winner(series):
    """Check if this line (could be anywhere)
    has 4 in a row"""
    count = 0
    prev = ' '
    for current_space in series:
        if current_space != ' ' and current_space == prev:
            count += 1
        elif current_space != ' ' and current_space != prev:
            count = 1
            prev = current_space
        elif current_space == ' ':
            prev = ' '
            count = 0
        
        if count == 4:
            return prev

def get_all_series(board):
    """Get all of the rows/cols/diags that could be winning streaks"""
    num_cols = len(board[0])
    num_rows = len(board)
    
    runs = []
    for row in range(0, num_rows):
        runs.append(board[row])
    for col in range(0, num_cols):
        runs.append(board[:,col])
        
    for offset in range(1-num_rows, num_cols):
        runs.append(np.diagonal(board, offset=offset))
        runs.append(np.diagonal(np.fliplr(board), offset=offset))
    # get all diags
    max_row_cols = max(num_rows, num_cols)

    return runs

def terminal(board):
    """Determine if a player has won"""
    num_cols = len(board[0])
    num_rows = len(board)
    if len(actions(board)) == 0:
        return 'c'
    
    series = get_all_series(board)
    for s in series:
        winner = check_series_for_winner(s)
        if winner is not None:
            return winner
    return ' '

def utility(board, player = 'x'):
    """check is a state a terminal state, return the utility if it is.
    None means not terminal"""
    
    winner = terminal(board)
    if winner == player:
        return +1
    if winner == flip_player(player):
        return -1
    if winner == 'c':
        return 0
    return None

def play_list_plays(moves, board, starting_player='x'):
    for move in moves:
        place_piece(board=board, player=starting_player, col=move)
        starting_player = flip_player(starting_player)

In [27]:
def pretty_board(board):
    for i in range(0,board.shape[1]+board.shape[1]+1):
        print('-',end='')
    print()
    for row in board:
        print('|', end='')
        for piece in row:
            print(piece, end='')
            print('|',end='')
        print() 
    for i in range(0,board.shape[1]+board.shape[1]+1):
        print('-',end='')
    print()

In [4]:

def play_game(player_x, player_o, board_shape = (6, 7), verbose=False):
    players_turn = 'x'
    
    board = empty_board(board_shape)
    while True:
        col = -1
        if players_turn == 'x':
            col = player_x(board, players_turn)
        else:
            col = player_o(board, players_turn)
        #place_piece(board, col, players_turn)
        board = result(board=board, player=players_turn, action=col)
        if verbose:
            print('Player: ', players_turn, 'placing in col', col)
        winner = terminal(board)
        if winner == 'c':
            return ('c', board)
        elif winner != ' ':
            return (players_turn, board)
        
        players_turn = flip_player(players_turn)


### Test Utility

In [5]:
board = empty_board()
pretty_board(board)
print(utility(board))
play_list_plays([3,2,3,1,3,0,3], board)
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
board = empty_board()
play_list_plays([0,1,1,2,3,2,2,5,3,3,3], board)
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
board = empty_board()
play_list_plays([0,1,1,2,3,2,2,5,3,3,3], board, starting_player='o')
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
board = empty_board(shape = (5,6))
play_list_plays([2,2,1,1,0,1,1,2,2,0,0], board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='x'))

---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
None
---------------
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | | |x| | | |
| | | |x| | | |
|o|o|o|x| | | |
---------------
x: 1 o: -1
---------------
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | |x|o| | | |
| |x|o|x| | | |
|x|o|o|x| |o| |
---------------
x: 1 o: -1
---------------
| | | | | | | |
| | | | | | | |
| | | |o| | | |
| | |o|x| | | |
| |o|x|o| | | |
|o|x|x|o| |x| |
---------------
x: -1 o: 1
x: None o: None


### Random V Random

In [6]:
x_ct = 0
o_ct = 0
c_ct = 0
games = []
for i in range(1000):
    game = play_game(agent_random, agent_random, verbose=False)
    winner = game[0]
    if winner == 'x':
        x_ct += 1
    elif winner == 'o':
        o_ct += 1
    elif winner == 'c':
        c_ct += 1
    games.append(game)
print('X won:', x_ct, '|O won', o_ct, '|Cat games', c_ct)

X won: 535 |O won 463 |Cat games 2


## Task 3: Minimax Search with Alpha-Beta Pruning [4 points]

### Implement the search starting from a given board and specifying the player.

In [58]:
import math
import random
DEBUG = 1 # 1 ... count nodes, 2 ... debug each node
COUNT = 0
def alpha_beta_search(board, player='x', actions=random_order_actions):
    global DEBUG, COUNT
    COUNT = 0
    
    value, move = max_value_ab(board, player, -math.inf, +math.inf, actions=actions)
    
    if DEBUG >= 1: print(f"Number of nodes searched: {COUNT}") 
        
    return value, move

def max_value_ab(board, player, alpha, beta, actions=actions):
    """Player's best move"""
    v = utility(board, player)
    global DEBUG, COUNT
    COUNT += 1
    if DEBUG >= 2:
        print("max: \n" + str(board) + str([alpha, beta, v])) 
    if v is not None: return v, None
    
    v, move = -math.inf, None
    
    moves = actions(board)
    for a in moves:
        v2, a2 = min_value_ab(result(board, player, a), player, alpha, beta, actions=actions)
        if v2 > v:
            v, move = v2, a
            alpha = max(alpha, v)
        if v >= beta:
            return v, move
    return v, move

def min_value_ab(board, player, alpha, beta, actions=actions):
    """opponent's best response"""
    global DEBUG, COUNT
    COUNT += 1
    
    #return utility if state is terminal state
    v = utility(board, player)
    if DEBUG >= 2: print("min: \n" + str(board) + str([alpha, beta, v]))
    if v is not None: return v, None
    
    v, move = +math.inf, None
    
    moves = actions(board)
    for a in moves:
        v2, a2 = max_value_ab(result(board, flip_player(player), a),player,alpha,beta, actions=actions)
        if v2 < v:
            v, move = v2, a
            beta = min(beta, v)
        if v <= alpha:
            return v, move
        
    return v, move

Experiment with some manually created boards (at least 5) to check if the agent spots wining opportunities.

### Find winning move

In [44]:
%%time
board = empty_board(shape = (4,5))
moves = [2,2,1,1,0,1,1,2]
play_list_plays(moves, board)
pretty_board(board)
DEBUG = 0
display(alpha_beta_search(board, player='x'))

-----------
| |x| | | |
| |o|o| | |
| |o|o| | |
|x|x|x| | |
-----------


(1, 3)

Wall time: 93 ms


### Play through this game (1)

In [45]:
%%time
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search(board, player='x'))

---------
| | | | |
| | | | |
| | | |o|
| |x|x|o|
|x|o|o|x|
---------


(1, 3)

Wall time: 208 ms


In [46]:
%%time
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search(board, player='o'))

---------
| | | | |
| | | |x|
| | | |o|
| |x|x|o|
|x|o|o|x|
---------


(-1, 1)

Wall time: 97 ms


In [47]:
%%time
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3,3]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search(board, player='x'))

---------
| | | |o|
| | | |x|
| | | |o|
| |x|x|o|
|x|o|o|x|
---------


(1, 2)

Wall time: 15 ms


In [48]:
%%time
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3,3,1]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search(board, player='o'))

---------
| | | |o|
| | | |x|
| |x| |o|
| |x|x|o|
|x|o|o|x|
---------


(-1, 1)

Wall time: 15 ms


In [49]:
%%time
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3,3,1,0]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search(board, player='x'))

---------
| | | |o|
| | | |x|
| |x| |o|
|o|x|x|o|
|x|o|o|x|
---------


(1, 1)

Wall time: 12 ms


In [50]:
%%time
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3,3,1,0,2]
play_list_plays(moves, board)
pretty_board(board)


---------
| | | |o|
| | | |x|
| |x|x|o|
|o|x|x|o|
|x|o|o|x|
---------
Wall time: 6 ms


TODO: Discuss why this is okay, and iven though it doesn't place in the obvious win column, it knows it will end up winning so it takes it.

### Play through this game (2)

In [51]:
%%time
board = empty_board(shape = (5,5))
play_list_plays(moves=[2,2,1,1,0,1,1,2,2],
                board=board,
                starting_player='x')
pretty_board(board)
DEBUG = 1
display(alpha_beta_search(board, player='o'))

-----------
| | | | | |
| |x|x| | |
| |o|o| | |
| |o|o| | |
|x|x|x| | |
-----------
Number of nodes searched: 25516


(1, 3)

Wall time: 2.29 s


### Next test

TODO: Add more tests

## How long does it take to decide the first move?

In [16]:
%%time
DEBUG = 1
board = empty_board(shape=(4,4))
display(alpha_beta_search(board, player='x'))

Number of nodes searched: 84003


(0, 3)

Wall time: 5.47 s


TODO: Add more tests on bigger empty boards

### Move ordering

My move ordering will explore the paths in the center before other moves. 
* actions_middle_first
 * Returns the middle actions first if they are available

In [19]:
def actions_middle_first(board):
    """Return the option of moves in the order starting with the middle"""
    moves = actions(board) # the moves in order
    num_cols = board.shape[1]
    new_order = []
    if board[0][num_cols//2] == ' ':
        new_order.append(num_cols//2)
    for i in range(1, num_cols//2+1):
        idx = (num_cols//2) + i
        if idx < num_cols and board[0][idx] == ' ':
            new_order.append(idx)
        idx = (num_cols//2) - i
        if idx >= 0 and board[0][idx] == ' ':
            new_order.append(idx)
    return new_order

board = empty_board(shape=(4,4))
print(actions_middle_first(board))
board = empty_board(shape=(4,5))
print(actions_middle_first(board))
board = empty_board(shape=(4,5))
play_list_plays([0,0,0,0], board)
print(actions_middle_first(board))

[2, 3, 1, 0]
[2, 3, 1, 4, 0]
[2, 3, 1, 4]


In [20]:
%%time
board = empty_board(shape=(4,4))
pretty_board(board)
display(alpha_beta_search(board, player='x',actions=random_order_actions))

---------
| | | | |
| | | | |
| | | | |
| | | | |
---------
Number of nodes searched: 88544


(0, 3)

Wall time: 5.73 s


In [52]:
%%time
board = empty_board(shape=(4,4))
display(alpha_beta_search(board, player='x',actions=actions_middle_first))

Number of nodes searched: 54505


(0, 2)

Wall time: 3.59 s


In [21]:
%%time
board = empty_board(shape=(4,5))
display(alpha_beta_search(board, player='x',actions=actions_middle_first))

Number of nodes searched: 5017045


(0, 2)

Wall time: 6min 9s


### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [140]:
# Your code/ answer goes here.
import sys
def mini_max_ab_agent(baord, player='x'):
    v, col = alpha_beta_search(board=board, player=player)

def mini_max_ab_agent_middle(board, player='x'):
    v, col = alpha_beta_search(board=board, player=player, actions=actions_middle_first)
    return col

DEBUG = 0

def play_games(N, agent_x, agent_o, shape):
    winnings =[0, 0, 0]
    for i in range(0, N):
        result_data = play_game(agent_x,
                                agent_o,
                                board_shape=shape)

        if result_data[0] == 'x': winnings[0]+=1
        if result_data[0] == 'o': winnings[1]+=1
        if result_data[0] == 'c': winnings[2]+=1
            
        sys.stderr.write('\rRuns: %d/%d' % (i+1, N))
        sys.stderr.flush()

    return winnings

winnings = play_games(100, mini_max_ab_agent_middle, agent_random, shape=(4,4))
print('x won:', winnings[0], 'o won', winnings[1], 'tie ', winnings[2])

Runs: 100/100

x won: 65 o won 0 tie  35


## Task 4: Heuristic Alpha-Beta Tree Search [3 points] 

### Heuristic evaluation function

this Heuristic will award 3 points for every open square on either side of 3 pieces in a row, and 2 points for each open side on 2 pieces in a row

In [65]:
def basic_heuristic(board, player='x'):
    series = get_all_series(board)
    util = utility(board, player)
    if util is not None: return util*100, True
    
    util = 0
    for s in series:
        as_string = ""
        as_string = as_string.join(s)
        # points for xxx
        
        idx = as_string.find(player*3)
        while idx >= 0:
            if idx-1 >= 0 and as_string[idx-1] == ' ':
                util += 3
            if idx >= 0 and idx+3 < len(as_string) and as_string[idx+3] == ' ':
                util += 3
            idx = as_string.find(player*3, idx+3)
            
        #points for xx
        idx = as_string.find(player*2)
        while idx >= 0:
            if idx-1 >= 0 and as_string[idx-1] == player:
                idx = as_string.find(player*2, idx+2)
                continue
            if idx+2 < len(as_string) and as_string[idx+2] == player:
                idx = as_string.find(player*2, idx+2)
                continue
            if idx-1 >= 0 and as_string[idx-1] == ' ':
                util += 2
            if idx > 0 and idx+2 < len(as_string) and as_string[idx+2] == ' ':
                util += 2
            idx = as_string.find(player*2, idx+2)
    return util, False

# Test 1
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3,3,1,0]
play_list_plays(moves, board)
pretty_board(board)
print('H(x) =', basic_heuristic(board, player='x'),'H(x) =',
      'H(o) =',basic_heuristic(board, player='o'))
board = empty_board(shape = (6,7))
board = empty_board(shape = (5,4))

# Test 2
moves = [0,1,1,2,3,3,2,3,3,3,1,0,2]
play_list_plays(moves, board)
pretty_board(board)
print('H(x) =', basic_heuristic(board, player='x'),'H(x) =',
      'H(o) =',basic_heuristic(board, player='o'))
board = empty_board(shape = (6,7))

# Test 3
moves = [3,3,4,4,2]
play_list_plays(moves, board)
pretty_board(board)
print('H(x) =', basic_heuristic(board, player='x'),'H(x) =',
      'H(o) =',basic_heuristic(board, player='o'))

---------
| | | |o|
| | | |x|
| |x| |o|
|o|x|x|o|
|x|o|o|x|
---------
H(x) = (7, False) H(x) = H(o) = (0, False)
---------
| | | |o|
| | | |x|
| |x|x|o|
|o|x|x|o|
|x|o|o|x|
---------
H(x) = (100, True) H(x) = H(o) = (-100, True)
---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |o|o| | |
| | |x|x|x| | |
---------------
H(x) = (6, False) H(x) = H(o) = (4, False)


### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [77]:
def alpha_beta_search_cutoff(baord, cutoff = None, player = 'x', H=basic_heuristic):
    global DEBUG, COUNT
    COUNT = 0
    
    value, move = max_value_ab_cutoff(board, player, -math.inf, +math.inf, 0, cutoff, H)
    
    if DEBUG >= 1:
        print(f"Number of nodes searched (cutoff = {cutoff}): {COUNT})")
        
    return value, move

def max_value_ab_cutoff(state, player, alpha, beta, depth, cutoff, H):
    """Player's best move"""
    global DEBUG, COUNT
    COUNT += 1
    
    # cut off and terminal test
    v, terminal = H(state, player)
    if((cutoff is not None and depth >= cutoff) or terminal): 
        if(terminal): alpha, beta = v, v
        if DEBUG >= 2: print(f"stopped at {depth}: {state} term: {terminal} eval: {v} [{alpha}, {beta}]" ) 
        return v, None
    
    v, move = -math.inf, None

    # check all possible actions in the state, update alpha and return move with the largest value
    for a in actions_middle_first(state):
        v2, a2 = min_value_ab_cutoff(result(state, player, a), player, alpha, beta, depth + 1, cutoff, H)
        if v2 > v:
            v, move = v2, a
            alpha = max(alpha, v)
        if v >= beta: return v, move
    return v, move

def min_value_ab_cutoff(state, player, alpha, beta, depth, cutoff, H):
    """opponent's best response."""
    global DEBUG, COUNT
    COUNT += 1
    
    # cut off and terminal test
    v, terminal = H(state, player)
    #if((cutoff is not None and depth >= cutoff) or terminal): 
    # always let the opponent make her move
    if(terminal): 
        alpha, beta = v, v
        if DEBUG >= 2:
            print(f"stopped at {depth}: {state} term: {terminal} eval: {v} [{alpha}, {beta}]" ) 
        return v, None
    
    v, move = +math.inf, None

    # check all possible actions in the state, update beta and return move with the smallest value
    for a in actions_middle_first(state):
        v2, a2 = max_value_ab_cutoff(result(state, flip_player(player), a), player, alpha, beta, depth + 1, cutoff, H)
        if v2 < v:
            v, move = v2, a
            beta = min(beta, v)
        if v <= alpha: return v, move
    
    return v, move
    
    

Experiment with the same manually created boards as above to check if the agent spots wining opportunities.

In [81]:
# Test 1
board = empty_board(shape = (5,4))
moves = [0,1,1,2,3,3,2,3,3,3,1,0]
play_list_plays(moves, board)
pretty_board(board)
print('H(x) =', basic_heuristic(board, player='x'),'H(x) =',
      'H(o) =',basic_heuristic(board, player='o'))

display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

# Test 2
board = empty_board(shape =(6,7))
moves = [3,3,3,3,3,3,4,2,4]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

---------
| | | |o|
| | | |x|
| |x| |o|
|o|x|x|o|
|x|o|o|x|
---------
H(x) = (7, False) H(x) = H(o) = (0, False)
Number of nodes searched (cutoff = 10): 34)


(100, 2)

---------------
| | | |o| | | |
| | | |x| | | |
| | | |o| | | |
| | | |x| | | |
| | | |o|x| | |
| | |o|x|x| | |
---------------
Number of nodes searched (cutoff = 10): 126901)


(17, 2)

In [84]:
# Test 2
board = empty_board(shape =(4,4))
moves = [0,0,1,1,2]
play_list_plays(moves, board)
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='o'))

---------
| | | | |
| | | | |
|o|o| | |
|x|x|x| |
---------
Number of nodes searched (cutoff = 10): 1959)


(2, 3)

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [91]:
%%time
board = empty_board(shape =(4,4))
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

---------
| | | | |
| | | | |
| | | | |
| | | | |
---------
Number of nodes searched (cutoff = 10): 8569)


(4, 2)

Wall time: 1.4 s


In [92]:
%%time
board = empty_board(shape =(4,5))
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

-----------
| | | | | |
| | | | | |
| | | | | |
| | | | | |
-----------
Number of nodes searched (cutoff = 10): 72211)


(4, 2)

Wall time: 13.4 s


In [93]:
%%time
board = empty_board(shape =(4,6))
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

-------------
| | | | | | |
| | | | | | |
| | | | | | |
| | | | | | |
-------------
Number of nodes searched (cutoff = 10): 54804)


(6, 3)

Wall time: 11.6 s


In [95]:
%%time
board = empty_board(shape =(6,6))
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

-------------
| | | | | | |
| | | | | | |
| | | | | | |
| | | | | | |
| | | | | | |
| | | | | | |
-------------
Number of nodes searched (cutoff = 10): 56515)


(8, 3)

Wall time: 13.5 s


In [96]:
%%time
board = empty_board(shape =(6,7))
pretty_board(board)
display(alpha_beta_search_cutoff(board, cutoff=10, player='x'))

---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
Number of nodes searched (cutoff = 10): 105987)


(8, 3)

Wall time: 28 s


### Forward Pruning

Add forward pruning to the cutoff search where you do not consider moves that have a low evaluation value after a shallow search 
(way smaller than the cuttoff value).

In [14]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [15]:
# Your code/ answer goes here.

### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function or different forward pruning) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [16]:
# Your code/ answer goes here.

## Challenge task [+ 1 bonus point]

Find another student and let your best agent play against the other student's best player. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [1 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

### Pure Monte Carlos Search

Implement Pure Monte Carlo Search and investigate how this search performs on the test boards that you have used above. 

#### Playout playouts function

In [102]:
def playout(board, action, player='x'):
    board = result(board, player, action)
    current_player = flip_player(player)
    
    while(True):
        u = utility(board, player)
        if u is not None: #if terminal return the utility
            return(u)
        
        col = agent_random(board, current_player)
        board = result(board, current_player, col)
        
        current_player = flip_player(current_player)

board = empty_board()
print(playout(board, 0))
print(playout(board, 0))
print(playout(board, 0))

-1
1
1


In [112]:
def playouts(board, action, player='x', N =100):
    return [playout(board, action, player) for i in range(N)]

board = empty_board()
p = playouts(board, 0)
print('--- first move 0 ---')
print(f"mean utility {np.mean(p)}")
print(f"win prob {sum(np.array(p) == +1)/len(p)}")
print(f"lose prob {sum(np.array(p) == -1)/len(p)}")
print('--- first move 3 ---')
p = playouts(board, 3)
print(f"mean utility {np.mean(p)}")
print(f"win prob {sum(np.array(p) == +1)/len(p)}")
print(f"lose prob {sum(np.array(p) == -1)/len(p)}")

--- first move 0 ---
mean utility 0.1
win prob 0.55
lose prob 0.45
--- first move 3 ---
mean utility 0.34
win prob 0.67
lose prob 0.33


In [133]:
def pure_monte_carlo_search(board, N=100, player='x'):
    
    global DEBUG
    
    action_options = actions(board)
    # find how many playouts each actions gets, force it to be at least 1
    n = max(1, N//len(action_options))
    if DEBUG >= 1:
        print(f"Actions: {action_options} ({n} playouts per actions)")
    
    # create a dictionary of the actions and their playout average value
    play_outcomes = {i:np.mean(playouts(board, i, player, N=n)) for i in action_options}
    
    if DEBUG >= 1:
        display(play_outcomes)
    
    action = max(play_outcomes, key=play_outcomes.get)
    return action

board = empty_board()
pretty_board(board)
DEBUG = 1
print("1000 playouts on empty board gets you:")
print(pure_monte_carlo_search(board, N=1000))
print("10 playouts on empty board gets you:")
print(pure_monte_carlo_search(board, N=10))

    

---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
100 playouts on empty board gets you:
Actions: [0, 1, 2, 3, 4, 5, 6] (142 playouts per actions)


{0: -0.1267605633802817,
 1: 0.028169014084507043,
 2: 0.2535211267605634,
 3: 0.30985915492957744,
 4: 0.23943661971830985,
 5: 0.09859154929577464,
 6: -0.1267605633802817}

3
10 playouts on empty board gets you:
Actions: [0, 1, 2, 3, 4, 5, 6] (1 playouts per actions)


{0: 1.0, 1: 1.0, 2: 1.0, 3: -1.0, 4: -1.0, 5: 1.0, 6: 1.0}

0


#### Monte Carlo Agents

In [141]:
def pmcs_1000(board, player):
    action = pure_monte_carlo_search(board, N=1000, player=player)
    return action

def pmcs_100(board, player):
    action = pure_monte_carlo_search(board, N=100, player=player)
    return action

winnings = play_games(100, pmcs_1000, pmcs_100, (6,7))
print('1000(x) won:', winnings[0], '100(o) won', winnings[1], 'tie ', winnings[2])

Runs: 100/100

1000(x) won: 95 100(o) won 5 tie  0


### Best First Move

How would you determine what the best first move is? You can use Pure Monte Carlo Search or any algorithms 
that you have implemented above.

In [116]:
for i in range(7):
    board = empty_board()
    p = playouts(board, i, N=1000)
    print(f"--- first move {i} ---")
    print(f"mean utility {np.mean(p)}")
    print(f"win prob {sum(np.array(p) == +1)/len(p)}")
    print(f"lose prob {sum(np.array(p) == -1)/len(p)}")
    

--- first move 0 ---
mean utility 0.026
win prob 0.511
lose prob 0.485
--- first move 1 ---
mean utility 0.08
win prob 0.539
lose prob 0.459
--- first move 2 ---
mean utility 0.144
win prob 0.57
lose prob 0.426
--- first move 3 ---
mean utility 0.288
win prob 0.644
lose prob 0.356
--- first move 4 ---
mean utility 0.155
win prob 0.576
lose prob 0.421
--- first move 5 ---
mean utility 0.082
win prob 0.539
lose prob 0.457
--- first move 6 ---
mean utility 0.017
win prob 0.507
lose prob 0.49


Here we use the playouts to determine how much each starting move wins if the rest of the moves are random across 1000 games. We see that moving at column 3, in the very middle does the best with a winning percentage of 64.4%